# Real-Time Stream Viewer (HTTP)
the following function responds to HTTP requests with the list of last 10 processed twitter messages + sentiments in reverse order (newest on top), it reads records from the enriched stream, take the recent 10 messages, and reverse sort them. the function is using nuclio context to store the last results and stream pointers for max efficiency.<br> 

The code is automatically converted into a nuclio (serverless) function and and respond to HTTP requests<br>

the example demonstrate the use of `%nuclio` magic commands to specify environment variables, package dependencies,<br>configurations, and to deploy functions automatically onto a cluster.


## Initialize nuclio emulation, environment variables and configuration
use `# nuclio: ignore` for sections that don't need to be copied to the function

In [11]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [12]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

### Set function configuration 
use a cron trigger with 5min interval and define the base image<br>
for more details check [nuclio function configuration reference](https://github.com/nuclio/nuclio/blob/master/docs/reference/function-configuration/function-configuration-reference.md)

In [13]:
%%nuclio config 
# ask for a specific (fixed) API port
# spec.triggers.web.kind = "http"
# spec.triggers.web.attributes.port = 30100
# define the function base docker image
spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


### Install required packages
`%nuclio cmd` allows you to run image build instructions and install packages<br>
Note: `-c` option will only install in nuclio, not locally

In [14]:
%nuclio cmd -c pip install v3io

## Nuclio function implementation
this function can run in Jupyter or in nuclio (real-time serverless)

In [15]:
import v3io.dataplane
import json
import os

def init_context(context):
    access_key = os.getenv('V3IO_ACCESS_KEY', None)
    setattr(context, 'container', os.getenv('V3IO_CONTAINER', 'bigdata'))
    setattr(context, 'stream_path', os.getenv('STOCKS_STREAM', 'stocks/stocks_stream'))
    
    v3io_client = v3io.dataplane.Client(endpoint=os.getenv('V3IO_API', None), access_key=access_key)
    resp = v3io_client.seek_shard(container=context.container, path=f'{context.stream_path}/0', seek_type='EARLIEST')
    setattr(context, 'next_location', resp.output.location)
    setattr(context, 'data', [])
    setattr(context, 'v3io_client', v3io_client) 
    setattr(context, 'limit', os.getenv('LIMIT', 10))

    
def handler(context, event):
    
    resp = context.v3io_client.get_records(container=context.container, path=f'{context.stream_path}/0', location=context.next_location, limit=context.limit)
    context.next_location = resp.output.next_location
    context.logger.info('location: %s', context.next_location)

    for rec in resp.output.records:
        rec_data = rec.data.decode('utf-8')
        rec_json = json.loads(rec_data)
        context.data.append(rec_json)

    context.data = context.data[-context.limit:]
    
    columns = [{'text': key, 'type': 'object'} for key in list(context.data[0].keys())]
    data = [list(item.values()) for item in context.data]
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]

    return response
                            

In [16]:
# nuclio: end-code

## Function invocation
the following section simulates nuclio function invocation and will emit the function results

In [ ]:
# create a test event and invoke the function locally
init_context(context)
event = nuclio.Event(body='')
handler(context, event)

## Deploy a function onto a cluster
the `%nuclio deploy` command deploy functions into a cluster, make sure the notebook is saved prior to running it !<br>check the help (`%nuclio help deploy`) for more information

In [18]:
from mlrun import code_to_function

environment_variables = {'V3IO_CONTAINER': 'bigdata',
                         'STOCKS_STREAM': 'stocks/stocks_stream'}

fn = code_to_function('stream-viewer',
                      kind='nuclio',
                      handler='handler')
fn.export('05-stream-viewer.yaml')
fn.set_envs(environment_variables)

> 2020-09-24 10:19:49,153 [info] function spec saved to path: 05-stream-viewer.yaml


In [19]:
addr = fn.deploy(project='stocks')

> 2020-09-24 10:19:49,173 [info] deploy started
[nuclio] 2020-09-24 10:19:50,251 (info) Build complete
[nuclio] 2020-09-24 10:19:54,287 (info) Function deploy complete
[nuclio] 2020-09-24 10:19:54,293 done updating stocks-stream-viewer, function address: 3.12.231.36:30100


In [20]:
# nuclio: ignore
# test the new API end point, take the address from the deploy log above
!curl {addr}

[{"columns": [{"text": "content", "type": "object"}, {"text": "time", "type": "object"}, {"text": "symbol", "type": "object"}, {"text": "link", "type": "object"}, {"text": "sentiment", "type": "object"}], "rows": [["By Shreyashi Sanyal and Devik Jain\n(Reuters) - The S&P 500 and Nasdaq indexes were set to open higher on Tuesday, with beaten-down shares of technology-related companies leading early gains, while Dow futures were subdued on uncertainty over more U.S. fiscal stimulus.\nAll three of Wall Street s main indexes started the week on the back foot as fears about a new round of lockdowns in Europe and a stalemate in Congress over the size and shape of another coronavirus-response bill dented hopes of a swift economic recovery.\nThe blue-chip Dow (DJI) closed Monday with its worst session in two weeks, while the benchmark S&P 500 (SPX) ended just under 9% down from its record high on Sept. 2, floating above correction territory.\nThe tech-heavy Nasdaq (IXIC) was largely unchanged,